In [1]:
import numpy as np
import gc
import pandas as pd
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from tensorflow import keras

In [2]:
df = pd.read_csv("DatasetTotal.csv", low_memory = False)

In [3]:
df = df.drop(columns = ['Timestamp'])
df = df.drop([0,1])

In [4]:
input_label = np.array(df.loc[:, df.columns != "Label"]).astype(np.float)

In [5]:
output_label = np.array(df["Label"])

In [6]:
out = []
for o in output_label:
    if(o == "Benign"):out.append(0)
    else: out.append(1)
output_label = out

In [7]:
scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(input_label)
input_label = scaler.transform(input_label)

In [8]:
input_label, output_label = shuffle(input_label, output_label)

In [9]:
inp_train,inp_test,out_train,out_test = train_test_split(input_label, input_label, test_size=0.2)

In [10]:
input_model = keras.layers.Input(shape = (78,))
enc = keras.layers.Dense(units = 64, activation = "relu", use_bias = True)(input_model)
enc = keras.layers.Dense(units = 36, activation = "relu", use_bias = True)(enc)
enc = keras.layers.Dense(units = 18, activation = "relu")(enc)
dec = keras.layers.Dense(units = 36, activation = "relu", use_bias = True)(enc)
dec = keras.layers.Dense(units = 64, activation = "relu", use_bias = True)(dec)
dec = keras.layers.Dense(units = 78, activation = "relu", use_bias = True)(dec)
auto_encoder = keras.Model(input_model, dec)

In [11]:
encoder = keras.Model(input_model, enc)
decoder_input = keras.layers.Input(shape = (18,))
decoder_layer = auto_encoder.layers[-3](decoder_input)
decoder_layer = auto_encoder.layers[-2](decoder_layer)
decoder_layer = auto_encoder.layers[-1](decoder_layer)
decoder = keras.Model(decoder_input, decoder_layer)

In [12]:
auto_encoder.compile(optimizer=keras.optimizers.Adam(lr=0.00025), loss = "mean_squared_error", metrics = ['accuracy'])

In [ ]:
train = auto_encoder.fit(x = np.array(inp_train), y = np.array(out_train),validation_split= 0.1, epochs = 10, verbose = 1, shuffle = True)

Epoch 1/10
 15047/117353 [==>...........................] - ETA: 1:38 - loss: 0.0152 - accuracy: 0.0857

In [14]:
input_label = encoder.predict(input_label).reshape(len(input_label), 18, 1)

In [15]:
inp_train,inp_test,out_train,out_test = train_test_split(np.array(input_label), np.array(output_label), test_size=0.2)

In [16]:
input_label = None
output_label = None
gc.collect()

6475

In [17]:
model = keras.Sequential([
        keras.layers.Conv1D(filters = 16, input_shape = (18,1), kernel_size = 3, padding = "same", activation = "relu", use_bias = True),
        keras.layers.MaxPool1D(pool_size = 3),
        keras.layers.Conv1D(filters = 8, kernel_size = 3, padding = "same", activation = "relu", use_bias = True),
        keras.layers.MaxPool1D(pool_size = 3),
        keras.layers.Flatten(),
        keras.layers.Dense(units = 2, activation = "softmax")
    ])

In [18]:
model.compile(optimizer= keras.optimizers.Adam(lr= 0.00025), loss="sparse_categorical_crossentropy", metrics=['accuracy'])

In [19]:
train = model.fit(x = inp_train, y = out_train, epochs = 10, validation_split = 0.1, shuffle = True, verbose = 1)

Epoch 1/10
7385/7385 [==============================] - 12s 1ms/step - loss: 0.5847 - accuracy: 0.7159 - val_loss: 0.5602 - val_accuracy: 0.7212
Epoch 2/10
7385/7385 [==============================] - 10s 1ms/step - loss: 0.5602 - accuracy: 0.7585 - val_loss: 0.5494 - val_accuracy: 0.7631
Epoch 3/10
7385/7385 [==============================] - 10s 1ms/step - loss: 0.5495 - accuracy: 0.7637 - val_loss: 0.5445 - val_accuracy: 0.7617
Epoch 4/10
7385/7385 [==============================] - 12s 2ms/step - loss: 0.5477 - accuracy: 0.7603 - val_loss: 0.5410 - val_accuracy: 0.7627
Epoch 5/10
7385/7385 [==============================] - 11s 2ms/step - loss: 0.5438 - accuracy: 0.7592 - val_loss: 0.5388 - val_accuracy: 0.7601
Epoch 6/10
7385/7385 [==============================] - 10s 1ms/step - loss: 0.5434 - accuracy: 0.7572 - val_loss: 0.5354 - val_accuracy: 0.7613
Epoch 7/10
7385/7385 [==============================] - 10s 1ms/step - loss: 0.5394 - accuracy: 0.7581 - val_loss: 0.5324 - val_ac

In [ ]:
res = np.array([np.argmax(resu) for resu in model.predict(inp_test)])

In [ ]:
print(confusion_matrix(out_test, res))
fpr, tpr, _ = roc_curve(out_test,  res)
print(str(fpr) + " | " + str(tpr))
print(roc_auc_score(out_test, res))